# Создание агента

Сами по себе языковые модели не могут выполнять действия — они просто выводят текст.
Одним из основных вариантов использования GigaChain является создание *агентов*.
Агенты — это системы, которые используют LLM в качестве механизма рассуждений, который определяет, какие действия нужно предпринять и какие входные данные для этих действий нужно использовать.
Результаты выполнения действий затем можно передать обратно агенту, чтобы он определил, нужно делать что-то еще или можно завершить работу.

В этом разделе приводится пример разработки агента, который может взаимодействовать с двумя инструментами: локальной базой данных и поисковой системой.
Вы сможете задавать этому агенту вопросы, наблюдать, как он вызывает инструменты, и вести с ним разговор.

## Основные понятия

В разеле затрагиваются следующие основные понятия:
- Работа с [языковыми моделями](/docs/concepts/#chat-models), в частности их способность вызывать инструменты.
- Создание [ретривера](/docs/concepts/#retrievers) для предоставления агенту конкретной информации.
- Использование [инструмента](/docs/concepts/#tools) для поиска информации в интернете.
- Использование [fгентов GigaGraph](/docs/concepts/#agents), которые используют LLM для обдумывания того, что делать, действуют на основе принятых решений.
<!--
- Отладка и трассировка вашего приложения с помощью [LangSmith](/docs/concepts/#langsmith)
-->

## Подготовка к разработке

### Jupyter-блокноты

Это руководство, как и большинство других в документации, использует [Jupyter-блокноты](https://jupyter.org/). Они отлично подходят для изучения работы с LLM-системами, так как предоставляют интерактивную среду для работы с руководствами и позволяют работать с непредвиденными ситуациями: недоступностью API, нетипичным выводом и другими.

Подробнее об установке jupyter -- в [официальной документации](https://jupyter.org/install).

### Установка

Для установки GigaChain выполните команды:

```{=mdx}
import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';
import CodeBlock from "@theme/CodeBlock";

<Tabs>
  <TabItem value="pip" label="Pip" default>
    <CodeBlock language="bash">pip install langchain</CodeBlock>
  </TabItem>
  <TabItem value="conda" label="Conda">
    <CodeBlock language="bash">conda install langchain -c conda-forge</CodeBlock>
  </TabItem>
</Tabs>
```


Подробнее об установке -- в разделе [Установка](https://developers.sber.ru/docs/ru/gigachain/get-started/installation).

<!--
### LangSmith

Многие приложения, которые вы создаете с помощью LangChain, будут содержать несколько шагов с многократными вызовами LLM.
По мере усложнения этих приложений становится важно иметь возможность инспектировать, что именно происходит внутри вашей цепочки или агента.
Лучший способ сделать это — с помощью [LangSmith](https://smith.langchain.com).

После регистрации по ссылке выше, убедитесь, что вы установили переменные среды для начала ведения журнала трассировок:

```shell
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="..."
```

Или, если вы работаете в ноутбуке, вы можете установить их с помощью:

```python
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
```
-->

## Определение инструментов

В первую очередь вам нужно создать инструменты, которые будет использовать агент.
В примере использованы два инструмента:
- Сторонний сервис [Tavily](/docs/integrations/tools/tavily_search) для поиска в интернете;
- Ретривер для извлечения данных из локального индекса, который нужно создать отдельно.

### Tavily

В GigaChain есть встроенный инструмент, который позволяет использовать поисковую систему Tavily в качестве инструмента.
Для работы с Tavily требуется API-ключ.
Пропустите этот шаг, если не хотите получать ключ и использовать сервис.

После создания API-ключа экспортируйте его с помощью команд:

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
search = TavilySearchResults(max_results=2)

In [4]:
search.invoke("what is the weather in SF")

[{'url': 'https://weather.com/weather/tenday/l/San Francisco CA USCA0987:1:US',
  'content': "Comfy & Cozy\nThat's Not What Was Expected\nOutside\n'No-Name Storms' In Florida\nGifts From On High\nWhat To Do For Wheezing\nSurviving The Season\nStay Safe\nAir Quality Index\nAir quality is considered satisfactory, and air pollution poses little or no risk.\n Health & Activities\nSeasonal Allergies and Pollen Count Forecast\nNo pollen detected in your area\nCold & Flu Forecast\nFlu risk is low in your area\nWe recognize our responsibility to use data and technology for good. recents\nSpecialty Forecasts\n10 Day Weather-San Francisco, CA\nToday\nMon 18 | Day\nConsiderable cloudiness. Tue 19\nTue 19 | Day\nLight rain early...then remaining cloudy with showers in the afternoon. Wed 27\nWed 27 | Day\nOvercast with rain showers at times."},
 {'url': 'https://www.accuweather.com/en/us/san-francisco/94103/hourly-weather-forecast/347629',
  'content': 'Hourly weather forecast in San Francisco, CA.

### Ретривер

Создадайте ретривер для извлечения данных.
Подробное объяснение процесса создания ретривера -- в разделе [Разработка RAG](/docs/tutorials/rag).

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [6]:
retriever.invoke("how to upload a dataset")[0]

Document(page_content='import Clientfrom langsmith.evaluation import evaluateclient = Client()# Define dataset: these are your test casesdataset_name = "Sample Dataset"dataset = client.create_dataset(dataset_name, description="A sample dataset in LangSmith.")client.create_examples(    inputs=[        {"postfix": "to LangSmith"},        {"postfix": "to Evaluations in LangSmith"},    ],    outputs=[        {"output": "Welcome to LangSmith"},        {"output": "Welcome to Evaluations in LangSmith"},    ],    dataset_id=dataset.id,)# Define your evaluatordef exact_match(run, example):    return {"score": run.outputs["output"] == example.outputs["output"]}experiment_results = evaluate(    lambda input: "Welcome " + input[\'postfix\'], # Your AI system goes here    data=dataset_name, # The data to predict and grade over    evaluators=[exact_match], # The evaluators to score the results    experiment_prefix="sample-experiment", # The name of the experiment    metadata={      "version": "1.0.0

После заполнения индекса, по которому будет выполняться поиск, вы можете преобразовать его в инструмент -- формат, который нужен для корректной работы агента.

In [7]:
from langchain.tools.retriever import create_retriever_tool

In [8]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

### Инструменты

Добавьте созданные инструменты в список, к которому будет обращаться агент.

In [9]:
tools = [search, retriever_tool]

## Использование языковых моделей

Ниже приводится пример, как использовать языковую модель для вызова инструментов.
GigaChain поддерживает различные языковые модели, которые могут заменять друг друга.

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs openaiParams={`model="gpt-4"`} />
```

In [10]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

Вызовите языковую модель, передав список сообщений.
По умолчанию ответом будет строка `content`.

In [11]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

Чтобы модель могла вызывать инструменты, передайте ей список инструментов с помощью метода `.bind_tools`.

In [12]:
model_with_tools = model.bind_tools(tools)

Обратитесь к модели с обычным сообщением и посмотрите, как она ответит.
Вы можете посмотреть как на поле `content`, так и на поле `tool_calls`.

In [13]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


Обратитесь к модели с входными данными, которые предполагают вызов инструмента.

In [14]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in SF'}, 'id': 'call_nfE1XbCqZ8eJsB8rNdn4MQZQ'}]


Из ответа модели видно, что нужно вызвать инструмент поиска Tavily Search.

Теперь, чтобы вызвать инструмент, нужно создать агента.

## Создание агента

Для создания агента используется библиотека [GigaGraph](/docs/concepts/#langgraph).
В примере ниже показано как создать агента с помощью высокоуровневого интерфейса, но GigaGraph также поддерживает работу с низкоуровневым API, который вы можете использовать если захотите изменить логику агента.

Инициализируйте агента с помощью LLM и инструментов.

Инициализируйте агента с помощью LLM и инструментов.

:::note

При создании агента используется `model`, а не `model_with_tools`.
Это связанно с тем, что метод `create_tool_calling_executor` самостоятельно вызывает `.bind_tools` для передачи инструментов.

:::

In [12]:
from langgraph.prebuilt import chat_agent_executor

agent_executor = chat_agent_executor.create_tool_calling_executor(model, tools)

## Запуск агента

Теперь вы можете попробовать обратиться к агенту с несколькими запросами.
На данном этапе агент работает без сохранения состояния и не запоминает результат предыдущих взаимодействий.
Также в результате работы агент вернет *итоговое* состояние,включающее все входные данные.
О том как получить только выходные данные вы можете узнать из других примеров ниже.

Сначала попробуйте обратиться к агенту с запросом, который не требует вызова инструментов.

In [16]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='1535b889-10a5-45d0-a1e1-dd2e60d4bc04'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 129, 'total_tokens': 139}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2c94c074-bdc9-4f01-8fd7-71cfc4777d55-0')]

<!--
Чтобы точно понять, что происходит под капотом (и убедиться, что инструмент не вызывается), мы можем взглянуть на [трассировку LangSmith](https://smith.langchain.com/public/28311faa-e135-4d6a-ab6b-caecf6482aaa/r).
-->

Теперь попробуйте передать запрос, который должен привести к вызову ретривера.

In [17]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="how can langsmith help with testing?")]}
)
response["messages"]

[HumanMessage(content='how can langsmith help with testing?', id='04f4fe8f-391a-427c-88af-1fa064db304c'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_FNIgdO97wo51sKx3XZOGLHqT', 'function': {'arguments': '{\n  "query": "how can LangSmith help with testing"\n}', 'name': 'langsmith_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 135, 'total_tokens': 157}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-51f6ea92-84e1-43a5-b1f2-bc0c12d8613f-0', tool_calls=[{'name': 'langsmith_search', 'args': {'query': 'how can LangSmith help with testing'}, 'id': 'call_FNIgdO97wo51sKx3XZOGLHqT'}]),
 ToolMessage(content="Getting started with LangSmith | 🦜️🛠️ LangSmith\n\nSkip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookQuick StartOn this pageGe

<!--
Посмотрим на [трассировку LangSmith](https://smith.langchain.com/public/853f62d0-3421-4dba-b30a-7277ce2bdcdf/r), чтобы увидеть, что происходит под капотом.
-->

Обратите внимание, что итоговое состояние, также содержит вызов инструмента и сообщение о результате вызова инструмента.

Теперь попробуйте передать запрос, который должен привести к вызову поискового инструмента.

In [18]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', id='e6b716e6-da57-41de-a227-fee281fda588'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_TGDKm0saxuGKJD5OYOXWRvLe', 'function': {'arguments': '{\n  "query": "current weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 134, 'total_tokens': 157}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-fd7d5854-2eab-4fca-ad9e-b3de8d587614-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_TGDKm0saxuGKJD5OYOXWRvLe'}]),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'lo

<!--
Мы можем проверить [трассировку LangSmith](https://smith.langchain.com/public/f520839d-cd4d-4495-8764-e32b548e235d/r), чтобы убедиться, что инструмент поиска вызывается эффективно.
-->

## Потоковая передача сообщений

В примерах выше вы видели, как можно вызвать агента с помощью метода `.invoke`, чтобы получить итоговый ответ.
Такой подход может занять некоторое время, если агент выполняет несколько шагов.
Для демонстрации промежуточных этапов, вы можете передавать сообщения по мере их возникновения.

In [19]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_50Kb8zHmFqPYavQwF5TgcOH8', 'function': {'arguments': '{\n  "query": "current weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 134, 'total_tokens': 157}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-042d5feb-c2cc-4c3f-b8fd-dbc22fd0bc07-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_50Kb8zHmFqPYavQwF5TgcOH8'}])]}}
----
{'action': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1714426906, \'localtime

## Потоковая передача токенов

Кроме потоковой передачи сообщений, также полезно передавать токены.
Это можно сделать с помощью метода `.astream_events`.

:::important

Метод `.astream_events` работает только в Python версии 3.11 или выше.

:::

In [22]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Назначено при создании агента с `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Назначено при создании агента с `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Пустое содержимой в контексте OpenAI означает,
            # что модель просит вызвать инструмент.
            # Поэтому выводится только непустое содержимое.
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: tavily_search_results_json with inputs: {'query': 'current weather in San Francisco'}
Done tool: tavily_search_results_json
Tool output was: [{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1714427052, 'localtime': '2024-04-29 14:44'}, 'current': {'last_updated_epoch': 1714426200, 'last_updated': '2024-04-29 14:30', 'temp_c': 17.8, 'temp_f': 64.0, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 23.0, 'wind_kph': 37.1, 'wind_degree': 290, 'wind_dir': 'WNW', 'pressure_mb': 1019.0, 'pressure_in': 30.09, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 50, 'cloud': 0, 'feelslike_c': 17.8, 'feelslike_f': 64.0, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 5.0, 'gust_mph': 27.5, 'gust_kph': 44.3}}"}, {'url': 'https://w

## Добавление памяти

Как уже упоминалось полученный агент не сохраняет данные о состоянии.
Это значит, что он не запоминает результаты предыдущих обращений.
Для добавления агенту памяти, вам нужно передать ему чекпоинтер.
В процессе передачи чекпоинтера, при вызове агента также нужно передать `thread_id`.
Таким образом агент будет знал, с какой ветки/разговора нужно продолжить работу.

In [24]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [25]:
agent_executor = chat_agent_executor.create_tool_calling_executor(
    model, tools, checkpointer=memory
)

config = {"configurable": {"thread_id": "abc123"}}

In [26]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 131, 'total_tokens': 142}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-607733e3-4b8d-4137-ae66-8a4b8ccc8d40-0')]}}
----


In [27]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob. How can I assist you further?', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 154, 'total_tokens': 167}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e1181ba6-732d-4564-b479-9f1ab6bf01f6-0')]}}
----


<!--
Пример [трассировки LangSmith](https://smith.langchain.com/public/fa73960b-0f7d-4910-b73d-757a12f33b2b/r)
-->

## Смотрите также

- [Документация GigaGraph](/docs/concepts/#langgraph).